# Deep Ensemble Calibration for Uncertainty Estimates

# Imports

In [ ]:
# system
import os 

# data
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from sklearn.metrics import accuracy_score

# deep learning 
import torch.backends.cudnn as cudnn
import torch
import torchvision
import torch.nn as nn

# custom helpers
import trainer
import uncertainty 

# Generics so I dont have to write more methods

In [ ]:
device = torch.device('cuda:0')
epochs = 30
learning_rate = 3e-4
weight_decay = 1e-5
step_period = 3
lr_decay = 0.95
num_workers = 6
batch_size = 128

model = torchvision.models.densenet121(pretrained=False)
model_lin = model.classifier.in_features
model.classifier = nn.Linear(model_lin, n_cls)
model = model.to(device)

# Helpers

In [ ]:
def loadPickle(fp):
    with open(fp, 'rb') as handle:
        return pickle.load(handle)

In [ ]:
def compute(model, fp, data):
    outputs = []
    for f in os.listdir(fp):
        full_model_path = os.path.join(fp, f)
        model.load_state_dict(torch.load(full_model_path))
        val_set = process.SkinSet(data['validation'])

        batch_total = 0.0
        labels = []
        predictions = [] 
    
        model.eval()
        with torch.no_grad():
            for idx, (inputs, targets) in enumerate(loader):
                inputs, targets = inputs.to(device), targets.to(device)
                preds = model(inputs)
                batch_total += 1
                labels.extend(targets.detach().cpu().numpy())
                predictions.extend(preds.detach().cpu().numpy())


        outputs.append(predictions, labels)
        
    return outputs

In [ ]:
data = loadPickle('./data/exp.pickle')
preds, labels = compute(model, './ensembles', data)

In [ ]:
mean = np.mean(preds, 0)
var = np.mean(np.var(prd,0),-1)

In [ ]:
n, bins, patches = plt.hist(mean, 50, density=True, facecolor='g', alpha=0.75)
plt.xlabel('Smarts')
plt.ylabel('Probability')
plt.title('Mean Histogram')
plt.grid(True)
plt.show()

In [ ]:
n, bins, patches = plt.hist(var, 50, density=True, facecolor='g', alpha=0.75)
plt.xlabel('Smarts')
plt.ylabel('Probability')
plt.title('Mean Histogram')
plt.grid(True)
plt.show()